In [1]:
import time
import pandas as pd
import numpy as np

import geopandas as gpd
from shapely.geometry import Point, Polygon

import warnings
warnings.filterwarnings('ignore')

import yaml
with open('config.yml') as f:
    config = yaml.load(f)

dwp = config['dcsedwp']
dcgis = config['dcgisprd']

import datetime as dt     
date = dt.datetime.today().strftime("%Y/%m/%d")
print(date)

2018/07/11


### Connect to DCGIS

In [2]:
import cx_Oracle
dsn_tns = cx_Oracle.makedsn(dcgis['host'], dcgis['port'], service_name=dcgis['service_name'])
dcgisprd = cx_Oracle.connect(dcgis['username'], dcgis['password'], dsn_tns)

In [3]:
blocks = gpd.read_file('data/Census_Blocks__2010.geojson')
blocks = blocks[['GEOID', 'BLOCK', 'BLKGRP', 'P0010001', 'SqMiles', 'ACRES', 'geometry']]

In [4]:
blks = blocks[['GEOID', 'P0010001', 'SqMiles']].set_index('GEOID')
blks['pop_density'] = blks['P0010001']*1.0/blks['SqMiles']
blks['tot_pop'] = blks['P0010001']
blks = blks.drop(['P0010001', 'SqMiles'], axis=1)

In [5]:
units = pd.read_sql('''
        select 
        a.address_id,
        count(*) as NUM_UNITS
        from MARDBA.VW_ADDRESS a
        left join MARDBA.VW_ADDRESSUNIT b on a.address_id = b.address_id
        where a.status='ACTIVE' 
        group by a.address_id
    ''', dcgisprd)

In [6]:
address = pd.read_sql('''
        select 
        a.address_id,
        case when res_type = 'RESIDENTIIAL' then 'RESIDENTIAL' else res_type end as RES_TYPE,
        a.latitude,
        a.longitude
        from MARDBA.VW_ADDRESS a
        where a.status='ACTIVE' 
    ''', dcgisprd)

In [7]:
address_units = address.merge(units, how='left', on='ADDRESS_ID')
## Check for duplicates after join
address_units.ADDRESS_ID.value_counts(ascending=False).head(n=10)

264191    1
26022     1
275884    1
265643    1
263594    1
269737    1
267688    1
290215    1
294309    1
81293     1
Name: ADDRESS_ID, dtype: int64

In [8]:
## Create geometry for MAR data
geometry = [Point(xy) for xy in zip(address_units.LONGITUDE.apply(float), address_units.LATITUDE.apply(float))]
crs = {'init': 'epsg:4326'}
points = gpd.GeoDataFrame(address_units, crs=crs, geometry=geometry)

address_blocks = gpd.sjoin(blocks, points, how='left', op='intersects')

In [9]:
unit_blocks = blks
for r in address_blocks[address_blocks.RES_TYPE.isnull()==False].RES_TYPE.unique():
    r_units = address_blocks[address_blocks.RES_TYPE==r]
    num_units = pd.DataFrame(r_units.groupby('GEOID').NUM_UNITS.sum())
    avg_units = pd.DataFrame(r_units.groupby('GEOID').NUM_UNITS.mean())
    med_units = pd.DataFrame(r_units.groupby('GEOID').NUM_UNITS.median())
    min_units = pd.DataFrame(r_units.groupby('GEOID').NUM_UNITS.count())
    max_units = pd.DataFrame(r_units.groupby('GEOID').NUM_UNITS.count())
    
    num_units.columns = ['num_'+r.lower().replace(' ', '_')]
    avg_units.columns = ['avg_'+r.lower().replace(' ', '_')]
    med_units.columns = ['med_'+r.lower().replace(' ', '_')]
    min_units.columns = ['min_'+r.lower().replace(' ', '_')]
    max_units.columns = ['max_'+r.lower().replace(' ', '_')]
    
    unit_blocks = unit_blocks.merge(num_units, how='left', left_index=True, right_index=True)
    unit_blocks = unit_blocks.merge(avg_units, how='left', left_index=True, right_index=True)
    unit_blocks = unit_blocks.merge(med_units, how='left', left_index=True, right_index=True)
    unit_blocks = unit_blocks.merge(min_units, how='left', left_index=True, right_index=True)
    unit_blocks = unit_blocks.merge(max_units, how='left', left_index=True, right_index=True)
    
unit_blocks = unit_blocks.drop(['pop_density', 'tot_pop'], axis=1)

In [10]:
unit_blocks.to_csv('data/address_units_to_blocks.csv.gz', compression = 'gzip')

## CAMA

### Residential

In [11]:
cama_res = pd.read_sql('''
    select
        ssl,
        bldg_num,
        ayb,
        eyb,
        case when ayb is null then to_number(substr(to_char(eyb),0,4))
            when ayb < 1754 then to_number(substr(to_char(eyb),0,4))
            when length(ayb) > 4 then to_number(substr(to_char(ayb),0,4))
            when ayb > 2017 then to_number(substr(to_char(ayb),0,4))
            else to_number(substr(to_char(ayb),0,4)) end as year_built,
        case when eyb is null then ayb else eyb end as alt_year_built,        
        yr_rmdl,
        cndtn_d
    from DCGIS.VW_CAMARES_OWNERPLY
    where ssl is not null and cndtn_d <> 'Default'
''', dcgisprd)

cama_res['CNDTN_D'] = cama_res.CNDTN_D.str.replace(' ', '')
cndtn = pd.DataFrame(pd.get_dummies(cama_res.CNDTN_D, prefix='ssl_cndtn'), index=cama_res.index)
cama_res = cama_res.merge(cndtn, how='left', left_index=True, right_index=True)

In [12]:
ssl_coords = pd.read_sql('''
    select
        ssl,
        median(latitude) as latitude,
        median(longitude) as longitude
    from MARDBA.VW_ADDRESS
    where latitude is not null and longitude is not null
        and ssl is not null
    group by ssl
''', dcgisprd)

In [13]:
cama = ['SSL', 'YEAR_BUILT', 'ssl_cndtn_Average', 'ssl_cndtn_Excellent', 'ssl_cndtn_Fair', 'ssl_cndtn_Good', 'ssl_cndtn_Poor', 'ssl_cndtn_VeryGood']
ssl_cama = ssl_coords.merge(cama_res[cama], how='left', on='SSL')

In [14]:
## Create geometry for CAMA Data
geometry = [Point(xy) for xy in zip(ssl_cama.LONGITUDE.apply(float), ssl_cama.LATITUDE.apply(float))]
crs = {'init': 'epsg:4326'}
points = gpd.GeoDataFrame(ssl_cama, crs=crs, geometry=geometry)

camares_blocks = gpd.sjoin(blocks, points, how='left', op='intersects')

In [15]:
avg_year_built = pd.DataFrame(camares_blocks.groupby('GEOID').YEAR_BUILT.mean())
med_year_built = pd.DataFrame(camares_blocks.groupby('GEOID').YEAR_BUILT.median())
min_year_built = pd.DataFrame(camares_blocks.groupby('GEOID').YEAR_BUILT.min())
max_year_built = pd.DataFrame(camares_blocks.groupby('GEOID').YEAR_BUILT.max())

avg_year_built.columns = ['avg_year_built']
med_year_built.columns = ['med_year_built']
min_year_built.columns = ['min_year_built']
max_year_built.columns = ['max_year_built']

ssl_g = ['ssl_cndtn_Average', 'ssl_cndtn_Excellent', 'ssl_cndtn_Fair', 'ssl_cndtn_Good', 'ssl_cndtn_Poor', 'ssl_cndtn_VeryGood']
avg_grade = pd.DataFrame(camares_blocks.groupby('GEOID')[ssl_g].mean())
avg_grade.columns = ssl_g

In [16]:
camares_blks = blks
camares_blks = camares_blks.merge(avg_year_built, how='left', left_index=True, right_index=True)
camares_blks = camares_blks.merge(med_year_built, how='left', left_index=True, right_index=True)
camares_blks = camares_blks.merge(min_year_built, how='left', left_index=True, right_index=True)
camares_blks = camares_blks.merge(max_year_built, how='left', left_index=True, right_index=True)
camares_blks = camares_blks.merge(avg_grade, how='left', left_index=True, right_index=True)
camares_blks = camares_blks.fillna(0)
camares_blks = camares_blks.drop(['pop_density', 'tot_pop'], axis=1)

### Commercial

In [17]:
cama_comm = pd.read_sql('''
    select
        ssl,
        bldg_num,
        ayb,
        eyb,
        case when ayb is null then to_number(substr(to_char(eyb),0,4))
            when ayb < 1754 then to_number(substr(to_char(eyb),0,4))
            when ayb > 2017 then to_number(substr(to_char(eyb),0,4))
            else to_number(substr(to_char(ayb),0,4)) end as year_built,
        case when eyb is null then ayb else eyb end as alt_year_built,        
        yr_rmdl,
        grade_d as cndtn_d
    from DCGIS.VW_CAMACOMM_OWNERPLY
    where ssl is not null and grade_d <> 'Default'
''', dcgisprd)

cama_comm['CNDTN_D'] = cama_comm.CNDTN_D.str.replace('+', '')
cama_comm['CNDTN_D'] = cama_comm.CNDTN_D.str.replace(' ', '')
cndtn = pd.DataFrame(pd.get_dummies(cama_comm.CNDTN_D, prefix='ssl_cndtn'), index=cama_res.index)
cama_comm = cama_comm.merge(cndtn, how='left', left_index=True, right_index=True)

In [18]:
cama = ['SSL', 'YEAR_BUILT', 'ssl_cndtn_Average', 'ssl_cndtn_Excellent', 'ssl_cndtn_Fair', 'ssl_cndtn_Good', 'ssl_cndtn_Poor', 'ssl_cndtn_VeryGood']
ssl_camacomm = ssl_coords.merge(cama_comm[cama], how='left', on='SSL')

In [19]:
## Create geometry for CAMA Commercial Data
geometry = [Point(xy) for xy in zip(ssl_camacomm.LONGITUDE.apply(float), ssl_camacomm.LATITUDE.apply(float))]
crs = {'init': 'epsg:4326'}
points = gpd.GeoDataFrame(ssl_camacomm, crs=crs, geometry=geometry)

camacomm_blocks = gpd.sjoin(blocks, points, how='left', op='intersects')

In [20]:
avg_year_built = pd.DataFrame(camacomm_blocks.groupby('GEOID').YEAR_BUILT.mean())
med_year_built = pd.DataFrame(camacomm_blocks.groupby('GEOID').YEAR_BUILT.median())
min_year_built = pd.DataFrame(camacomm_blocks.groupby('GEOID').YEAR_BUILT.min())
max_year_built = pd.DataFrame(camacomm_blocks.groupby('GEOID').YEAR_BUILT.max())

avg_year_built.columns = ['avg_year_built']
med_year_built.columns = ['med_year_built']
min_year_built.columns = ['min_year_built']
max_year_built.columns = ['max_year_built']

ssl_g = ['ssl_cndtn_Average', 'ssl_cndtn_Excellent', 'ssl_cndtn_Fair', 'ssl_cndtn_Good', 'ssl_cndtn_Poor', 'ssl_cndtn_VeryGood']
avg_grade = pd.DataFrame(camacomm_blocks.groupby('GEOID')[ssl_g].mean())
avg_grade.columns = ssl_g

In [21]:
camacomm_blks = blks
camacomm_blks = camacomm_blks.merge(avg_year_built, how='left', left_index=True, right_index=True)
camacomm_blks = camacomm_blks.merge(med_year_built, how='left', left_index=True, right_index=True)
camacomm_blks = camacomm_blks.merge(min_year_built, how='left', left_index=True, right_index=True)
camacomm_blks = camacomm_blks.merge(max_year_built, how='left', left_index=True, right_index=True)
camacomm_blks = camacomm_blks.merge(avg_grade, how='left', left_index=True, right_index=True)
camacomm_blks = camacomm_blks.fillna(0)
camacomm_blks = camacomm_blks.drop(['pop_density', 'tot_pop'], axis=1)

## Combine CAMA Res and Comm

In [22]:
cama_blks = camares_blks.merge(camacomm_blks, how='outer', left_index=True, right_index=True, suffixes=['_res', '_comm'])

In [23]:
cama_blks.to_csv('data/cama_to_blocks.csv.gz', compression = 'gzip')